In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import matplotlib.pyplot as plt

In [6]:
# Define paths to directories containing feature files
porn_dir = r'A:\AI DB\LSPD\Videos\Features_Porn'
normal_dir = r'A:\AI DB\LSPD\Videos\Features_Normal'

In [7]:
# Load features and labels for porn
features_porn = np.concatenate([np.load(file, allow_pickle=True) for file in porn_files])
num_samples_porn = len(features_porn)
labels_porn = np.ones(num_samples_porn)  # Create labels filled with 1 indicating "porn"

# Load features and labels for normal
features_normal = np.concatenate([np.load(file, allow_pickle=True) for file in normal_files])
num_samples_normal = len(features_normal)
labels_normal = np.zeros(num_samples_normal)  # Create labels filled with 0 indicating "normal"


In [8]:
print(" porn features : ",features_porn.shape)
print(" normal features : ",features_normal.shape)

 visual audio data shape :  (15, 9, 2176)


In [ ]:
# Concatenate features and labels
features = np.concatenate((features_porn, features_normal), axis=0)
labels = np.concatenate((labels_porn, labels_normal), axis=0)


In [ ]:
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
# Define the LSTM model with specified hyperparameters
model = Sequential([
    LSTM(10, input_shape=(10, 2176), return_sequences=True, activation='tanh',
         recurrent_activation='sigmoid', dropout=0.0, recurrent_dropout=0.0,
         unroll=False, use_bias=True),
    LSTM(10, activation='tanh', recurrent_activation='sigmoid', dropout=0.0,
         recurrent_dropout=0.0, unroll=False, use_bias=True),
    Dense(1, activation='sigmoid')
])

# Compile the model with specified optimizer and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define model checkpoint
checkpoint_filepath = r'A:\AI DB\LSPD\models\LSTMmodel.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, shuffle=True,
                    validation_data=(X_val, y_val),
                    callbacks=[model_checkpoint_callback])

In [ ]:
# Evaluate the model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Generate predictions using the LSTM model
lstm_predictions = model.predict(X_test)

# Convert probabilities to predicted classes
lstm_pred_classes = (lstm_predictions > 0.5).astype(int)

# Get true classes
true_classes = y_test

# Create a confusion matrix
confusion_matrix_lstm = confusion_matrix(true_classes, lstm_pred_classes)
normalized_confusion_matrix_lstm = confusion_matrix_lstm.astype('float') / confusion_matrix_lstm.sum(axis=1)[:, np.newaxis]

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(normalized_confusion_matrix_lstm, annot=True, fmt='.2f', cmap='YlGnBu')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix - LSTM Model')
plt.show()

# Calculate accuracy
accuracy_lstm = accuracy_score(true_classes, lstm_pred_classes)
print("Accuracy (LSTM Model):", accuracy_lstm)

# Calculate precision, recall, and F1-score
precision_lstm, recall_lstm, f1_score_lstm, _ = precision_recall_fscore_support(true_classes, lstm_pred_classes)

# Print precision, recall, and F1-score for each class
for i, label in enumerate(["Normal", "Porn"]):
    print(f"{label} - Precision: {precision_lstm[i]}, Recall: {recall_lstm[i]}, F1-score: {f1_score_lstm[i]}")

# Print classification report
classification_report_lstm = classification_report(true_classes, lstm_pred_classes, target_names=["Normal", "Porn"])
print("Classification Report - LSTM Model:\n", classification_report_lstm)